## AI Governance Lab: Building Secure HR Agents with Databricks

In this hands-on lab, you'll learn how to build AI agents that respect data governance policies using Databricks Unity Catalog. We'll create an HR Analyst agent that can answer questions about employees, compensation, and policies while automatically enforcing security controls through view-based access control and data classification.

### Why This Matters
As organizations deploy AI agents with access to sensitive data, governance becomes critical. Without proper controls, an AI agent could inadvertently expose confidential information like salaries, SSNs, or HR cases. This lab demonstrates how to build agents that are both helpful and secure using a layered security approach with data classifications.

### Prerequisites Completed ✓
- Unity Catalog enabled
- HR tables created and classified in `clientcare.hr_data`:
  - `employee_records` (Confidential) - Contains PII
  - `compensation_data` (Restricted) - Contains salary information
  - `performance_reviews` (Confidential) - Contains employee assessments
  - `hr_cases` (Restricted) - Contains sensitive HR matters
  - `public_policies` (Public) - Available to all
  - `internal_procedures` (Internal) - For employees only
- Group `hr_data_analysts` was pre-created in the environment

### Lab Structure
**Lab 1: Building the Governance Foundation**
1. Apply Data Classifications to tables
2. Create Classification-Aware Views with anonymization and filtering
3. Create Security Group for HR analytics access
4. Grant Group Permissions on views and functions
5. Implement Table-Level Column Masking for SSN
6. Build Secure Tools (UC Functions) for the agent
7. Test the complete governance stack

**Lab 2: Implementing the AI Agent**
1. Build the HR Analyst agent using the secure tools from Lab 1
2. Register the agent model in MLflow
3. Deploy agent as model endpoint
4. Verify agent permissions through group membership
5. Test agent with governance-aware queries
6. Evaluate traces to validate security controls

### **Initial Setup and Data Verification**

Let's start by verifying our environment and examining the data classifications we've already applied.

_Background for those new to Databricks_

- We use `spark.sql()` for all governance OPERATIONS (CREATE VIEW, GRANT, etc.)
- We use `pandas.DataFrame()` for showing results in a nice format
- We use the Unity Catalog SDK (`WorkspaceClient`) for verification and reading metadata

**Implementing Unity Catalog Governance: Available Methods**

1. **Pure SQL** - The standard approach for production environments 
2. **Spark SQL in Python** - SQL commands as strings in Python (what we'll use)
3. **PySpark DataFrame API** - The Pythonic approach for queries
4. **Databricks SDK** - For reading metadata and automation
5. **REST API** - For external integrations and CI/CD pipelines
6. **Terraform** - Infrastructure as Code approach for version-controlled, repeatable deployments

In this lab, we'll primarily use **Spark SQL in Python** (#2) for governance operations and the **SDK** (#4) for verification.

In [0]:
# --------------------------------------------
# CLEANUP: RESET ALL GOVERNANCE OBJECTS
# --------------------------------------------
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

catalog_name = "clientcare" 
schema_name = "hr_data"

spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

print("🔄 CLEANING UP ANY EXISTING GOVERNANCE OBJECTS...")

# Remove any direct table masks (if applied from previous runs)
try:
    spark.sql("ALTER TABLE employee_records ALTER COLUMN ssn DROP MASK")
    spark.sql("ALTER TABLE employee_records ALTER COLUMN phone DROP MASK")
    spark.sql("ALTER TABLE employee_records ALTER COLUMN email DROP MASK")
    spark.sql("ALTER TABLE compensation_data ALTER COLUMN base_salary DROP MASK")
    spark.sql("ALTER TABLE compensation_data ALTER COLUMN bonus DROP MASK")
    print("✓ Removed any existing column masks")
except Exception as e:
    print(f"Note: No existing masks to remove")

# Drop existing views
views_to_drop = [
    "data_analyst_view",
    "admin_hr_view",
    "hr_public_view",
    "hr_internal_view", 
    "hr_confidential_view",
    "hr_restricted_view",
    "hr_unified_view"
]

for view in views_to_drop:
    try:
        spark.sql(f"DROP VIEW IF EXISTS {view}")
        print(f"✓ Dropped view {view}")
    except Exception as e:
        print(f"Note: View {view} didn't exist")

print("\n✅ Cleanup complete! Ready to build governance from scratch.")

# Show current state of raw data that needs protection
print("\n⚠️ Current UNPROTECTED data:")
display(spark.sql("""
    SELECT 
        e.employee_id,
        e.first_name,
        e.ssn,              -- Needs masking!
        e.department,
        c.base_salary,      -- Needs masking!
        c.bonus            -- Needs masking!
    FROM employee_records e
    JOIN compensation_data c ON e.employee_id = c.employee_id
    LIMIT 5
"""))

In [0]:
# Set our working environment

from databricks.sdk import WorkspaceClient
from pyspark.sql import SparkSession
import pandas as pd

w = WorkspaceClient()# Read-only for most operations
# Cannot set masks or row filters via SDK

catalog_name = "clientcare" 
schema_name = "hr_data"

# Verify all tables are loaded
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")


In [0]:
# This demonstrates the current security risk - without proper governance controls,
# all sensitive data (SSNs, salaries, bonuses) is fully exposed to any user with table access
# In a real production environment, this would be a major compliance violation

display(spark.sql("""
   SELECT 
       e.employee_id,
       e.first_name,
       e.last_name,
       e.department,
       e.ssn,
       e.phone,
       e.email,
       e.hire_date,
       c.base_salary,
       c.bonus,
       c.stock_options
   FROM employee_records e
   JOIN compensation_data c ON e.employee_id = c.employee_id
   ORDER BY e.employee_id
   LIMIT 10
"""))

### **Apply Data Classifications**

First, we'll tag our tables with sensitivity levels. Table properties are metadata tags that help classify data sensitivity without changing the actual data structure:

- Metadata key-value pairs attached to tables
- Used by governance tools to understand data sensitivity
- Don't affect table structure or data
- Can be queried programmatically for compliance

Our data classification schema:
 - Public: Anyone can access (company policies)
 - Internal: Employees only (procedures)
 - Confidential: Limited access (employee records, reviews)
 - Restricted: Highly sensitive (compensation, HR cases)

In [0]:
# Define classification scheme
tables_config = [
    ("employee_records", "Confidential", "true"),     # Names, SSN, contact info
    ("compensation_data", "Restricted", "true"),      # Salaries and bonuses
    ("performance_reviews", "Confidential", "true"),  # Employee evaluations
    ("hr_cases", "Restricted", "true"),               # Sensitive HR investigations
    ("public_policies", "Public", "false"),           # Company handbook
    ("internal_procedures", "Internal", "false")      # HR processes
]

# Apply classifications using spark.sql
for table_name, classification, has_pii in tables_config:
    spark.sql(f"""
        ALTER TABLE {table_name} SET TBLPROPERTIES (
            'data_classification' = '{classification}',
            'contains_pii' = '{has_pii}',
            'governance_enabled' = 'true'
        )
    """)
    print(f"✓ Classified {table_name} as {classification}")


In [0]:
# Verify using SDK to read properties
verification_data = []
for table_name, expected_class, expected_pii in tables_config:
    try:
        # Get table info using SDK
        table = w.tables.get(full_name=f"{catalog_name}.{schema_name}.{table_name}")
        
        # Properties are read-only via SDK
        properties = table.properties if table.properties else {}
        
        verification_data.append({
            "Table": table_name,
            "Classification": properties.get('data_classification', 'Not set'),
            "Contains PII": properties.get('contains_pii', 'Not set'),
            "Governance Enabled": properties.get('governance_enabled', 'Not set')
        })
    except Exception as e:
        verification_data.append({
            "Table": table_name,
            "Classification": f"Error: {str(e)}",
            "Contains PII": "-",
            "Governance Enabled": "-"
        })

display(pd.DataFrame(verification_data))


**Agent Permissions & Agent Access Requirements**
Now that we've classified our data, let's design the right access level for our HR analytics agent through purpose-built views.

**Key Decision:** The agent needs a specialized view - "Data Analyst View" - that provides analytical capabilities while protecting individual privacy.

What the Agent Needs vs. Doesn't Need:
| Data Type | HR Admin Sees | Manager Sees | Data Analyst Agent | Why? |
|-----------|---------------|--------------|------------------|------|
| **Names** | ✅ John Smith | ✅ John Smith | ❌ Anonymous IDs | Prevents bias, protects privacy |
| **SSN** | ✅ 123-45-6789 | ❌ Hidden | ❌ Hidden | No analytical value |
| **Salary** | ✅ $120,000 | ❌ Hidden | ✅ $120,000 | Needed for accurate statistics |
| **Department** | ✅ Engineering | ✅ Engineering | ✅ Engineering | Needed for grouping |
| **Performance** | ✅ 4.5 | ✅ 4.5 | ✅ 4.5 | Needed for correlation analysis |
| **Email** | ✅ john@company.com | ✅ john@company.com | ❌ Hidden | No analytical value |
| **Phone** | ✅ 555-1234 | ✅ 555-1234 | ❌ Hidden | No analytical value |

The Agent's Mission:
Answer questions like:
- "What's the salary distribution in Engineering?"
- "Is there pay equity across departments?"
- "What's the correlation between performance and compensation?"

WITHOUT being able to answer:
- "What's John Smith's salary?"
- "Who are the top 5 highest paid employees?"
- "Show me SSNs for employees making over $100k"

### **Create Classification-Aware Views**

Now we'll create views that automatically provide appropriate data access based on our classifications. These views serve as the foundation for access control in Databricks.

**Why Views for Access Control:**
- **Simplicity**: Instead of complex masking rules for every possible user type, create purpose-built views
- **Clarity**: Each view has a clear business purpose and user type
- **Maintainability**: Easier to understand and modify than intricate permission matrices
- **Performance**: Views are optimized SQL - no runtime masking overhead
- **Security Layer**: In Databricks, views act as your security layer since we grant permissions directly to principals

**Our View Strategy:**
We're building two focused views that align with our actual use cases:
1. **Data Analyst View**: Anonymous IDs + full compensation data for statistical analysis (designed for AI agents and analysts)
2. **Admin View**: Complete unmasked access for HR administrators

Rather than trying to anticipate every possible access pattern, we're creating views that solve our specific governance requirements efficiently.

**📝 Note: Why No Roles?**
If you're coming from traditional databases, you might expect to use `CREATE ROLE`. Databricks Unity Catalog works differently:
- **No CREATE ROLE**: This SQL statement doesn't exist in Databricks
- **Groups**: Managed at the account/workspace level (we'll use the pre-created `hr_data_analysts` group)
- **Direct Grants**: Permissions are granted to users, groups, or service principals
- **Why it works**: The views themselves act as your "roles" by defining what each access level sees

Next, we'll configure the `hr_data_analysts` group with appropriate permissions on these views.

In [0]:
# DATA ANALYST VIEW - For statistical analysis (will be assigned to agent service principal in Lab 2)
# Key features: Anonymous employee IDs, full salary access, excludes Legal dept
spark.sql("""
CREATE OR REPLACE VIEW data_analyst_view AS
SELECT 
    CONCAT('EMP_', LPAD(e.employee_id, 6, '0')) as anonymous_id,  -- EMP_000001 format
    e.department,
    YEAR(e.hire_date) as hire_year,                            -- Year only for better anonymization
    c.base_salary,                                              -- Full salary for analytics
    c.bonus,
    c.stock_options,
    YEAR(c.effective_date) as comp_year,
    pr.rating,
    QUARTER(pr.review_date) as review_quarter,
    YEAR(pr.review_date) as review_year
    -- Removed pr.comments to prevent identifying information
FROM employee_records e
LEFT JOIN compensation_data c ON e.employee_id = c.employee_id
LEFT JOIN performance_reviews pr ON e.employee_id = pr.employee_id
WHERE e.department != 'Legal'  -- Exclude Legal for compliance reasons
""")
print("✓ Created data_analyst_view: Anonymous IDs + full compensation data (enhanced anonymization)")

In [0]:
# ADMIN VIEW - Complete access for HR administrators
spark.sql("""
CREATE OR REPLACE VIEW admin_hr_view AS
SELECT 
    e.employee_id,
    e.first_name,
    e.last_name,
    e.department,
    e.email,
    e.ssn,
    e.phone,
    e.hire_date,
    e.manager_id,
    c.base_salary,
    c.bonus,
    c.stock_options,
    c.effective_date,
    pr.rating,
    pr.review_date,
    pr.comments,
    hr.case_id,
    hr.case_type,
    hr.status,
    hr.opened_date,
    hr.closed_date,
    hr.severity,
    hr.description
FROM employee_records e
LEFT JOIN compensation_data c ON e.employee_id = c.employee_id
LEFT JOIN performance_reviews pr ON e.employee_id = pr.employee_id
LEFT JOIN hr_cases hr ON e.employee_id = hr.employee_id
""")

print("✓ Created admin_hr_view: Complete access to all HR data")

### **Configure Group Permissions**

Now we'll grant permissions to the pre-created `hr_data_analysts` group. This group will be used by:
- Your user account (for testing in this lab)
- The AI agent's service principal (in production deployments)
- Any data analysts who need access to anonymized HR data

This implements the principle of least privilege - the group only gets access to anonymized, aggregated data, never raw employee records.

In [0]:
# Verify the hr_data_analysts group exists
groups_df = spark.sql("SHOW GROUPS")
display(groups_df)
print("✅ Confirmed: hr_data_analysts group is available")

In [0]:
# Confirm views in the schema
print("📋 Verifying our governance views were created successfully...")
print("\nViews in clientcare.hr_data:")
views = spark.sql("SHOW VIEWS IN clientcare.hr_data")
display(views)

print("\n✅ You should see:")
print("   - data_analyst_view (for agents and analysts)")
print("   - admin_hr_view (for HR administrators)")

In [0]:
# Grant permissions to the hr_data_analysts group using fully qualified names
catalog_name = "clientcare"
schema_name = "hr_data"
group_name = "hr_data_analysts"

# Catalog and schema access
spark.sql(f"GRANT USAGE ON CATALOG {catalog_name} TO `{group_name}`")
spark.sql(f"GRANT USAGE ON SCHEMA {catalog_name}.{schema_name} TO `{group_name}`")

# View access with fully qualified name
spark.sql(f"GRANT SELECT ON VIEW {catalog_name}.{schema_name}.data_analyst_view TO `{group_name}`")

print(f"✅ Granted permissions to {group_name} group:")
print(f"   - USAGE on catalog: {catalog_name}")
print(f"   - USAGE on schema: {catalog_name}.{schema_name}")
print(f"   - SELECT on view: {catalog_name}.{schema_name}.data_analyst_view")
print("\n📝 Note: We'll grant EXECUTE on UC functions after creating them later in this lab")

### **Column Masking**
**Why implement this when we already have views?**

Views can be bypassed if users gain direct table access. Column masking at the table level cannot be bypassed - it's enforced by Unity Catalog on every query, regardless of how the data is accessed.

**How Table-Level Security Works:**
- **Column Masking**: Functions run automatically transforming sensitive data based on who's viewing it
- **Row-Level Security**: Functions can also control which rows users see (we already handled Legal department filtering in our views, but this could be done at the table level too)
- Unity Catalog enforces these at the storage layer - users cannot bypass them
- Different users see different versions of the same data

**Implementation:**
We'll mask the SSN field to show data transformation in action. We could also add row-level filtering (like excluding Legal department records), but since we already handled that requirement in our views, we'll focus on column masking.

**Production Value:**
If someone accidentally grants direct table access or bypasses your views, table-level controls still protect sensitive data automatically.

In [0]:
# Implement column masking for SSN field - Defense in depth security
print("🔐 Implementing column masking for SSN field...")

# Create column masking function for SSN
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog_name}.{schema_name}.mask_ssn(ssn_value STRING)
RETURNS STRING
RETURN CASE 
    WHEN is_account_group_member('admins') THEN ssn_value
    WHEN is_account_group_member('hr_data_analysts') THEN 'ANALYTICS_MASKED'
    ELSE CONCAT('***-**-', RIGHT(ssn_value, 4))
END
""")

# Apply the masking function to the SSN column
spark.sql(f"""
ALTER TABLE {catalog_name}.{schema_name}.employee_records 
ALTER COLUMN ssn 
SET MASK {catalog_name}.{schema_name}.mask_ssn
""")

print("✅ Column masking applied to SSN field")
print("🔐 Defense in Depth: SSN now masked at table level")
print("   - Admins see full SSN")
print("   - hr_data_analysts group sees 'ANALYTICS_MASKED'") 
print("   - Other users see '***-**-1234' format")

In [0]:
# Validate column masking implementation
print("🧪 Testing SSN column masking...")

print("\n👤 Current user view:")
display(spark.sql(f"""
SELECT employee_id, first_name, last_name, ssn, department 
FROM {catalog_name}.{schema_name}.employee_records 
LIMIT 5
"""))


### **Build Secure Tools for the Agent**
Now we'll create Unity Catalog functions that our AI agent will use to query HR data. These functions act as the interface between the agent and our governed data.

**Why UC Functions for Agent Tools?**
- **Governed Access**: Functions respect the same permissions as the calling principal
- **Input Validation**: Can validate and sanitize agent inputs before querying
- **Query Templates**: Provide pre-defined, safe query patterns the agent can use
- **Audit Trail**: All function calls are logged for compliance
- **Performance**: Functions can be optimized with proper indexing and caching

**Our Agent Tool Strategy:**
We'll create two general-purpose functions that:
1. Work exclusively with the anonymized `data_analyst_view`
2. Return aggregated results that can answer various HR questions
3. Prevent the agent from writing arbitrary SQL
4. Maintain employee privacy through anonymous IDs

These functions will be the ONLY way our agent interacts with the data, ensuring consistent governance.

In [0]:
# TOOL 1: Performance & Retention Analytics
print("🔧 Creating performance analytics function...")

spark.sql(f"""
    CREATE OR REPLACE FUNCTION
    {catalog_name}.{schema_name}.analyze_performance()
    RETURNS TABLE (
        department STRING,
        avg_rating DOUBLE,
        min_rating DOUBLE,
        max_rating DOUBLE,
        employee_count INT,
        avg_tenure_years DOUBLE
    )
    COMMENT 'HR Analytics: Basic performance metrics by department'
    RETURN (
        SELECT 
            department,
            AVG(rating) as avg_rating,
            MIN(rating) as min_rating,
            MAX(rating) as max_rating,
            COUNT(DISTINCT anonymous_id) as employee_count,
            AVG(YEAR(CURRENT_DATE()) - hire_year) as avg_tenure_years
        FROM {catalog_name}.{schema_name}.data_analyst_view
        WHERE rating IS NOT NULL
        GROUP BY department
    );
""")

print("✓ Created analyze_performance function")
print("  - Returns: avg/min/max ratings, employee count, avg tenure by department")
print("  - Works with: anonymized data only")

In [0]:
# TOOL 2: Department & Compensation Analytics
print("🔧 Creating operations analytics function...")

spark.sql(f"""
    CREATE OR REPLACE FUNCTION
    {catalog_name}.{schema_name}.analyze_operations()
    RETURNS TABLE (
        department STRING,
        employee_count INT,
        avg_salary DOUBLE,
        avg_bonus DOUBLE,
        avg_total_comp DOUBLE,
        avg_stock_options INT
    )
    COMMENT 'HR Analytics: Department compensation and operational metrics'
    RETURN (
        SELECT 
            department,
            COUNT(DISTINCT anonymous_id) as employee_count,
            AVG(base_salary) as avg_salary,
            AVG(bonus) as avg_bonus,
            AVG(base_salary + bonus) as avg_total_comp,
            AVG(stock_options) as avg_stock_options
        FROM {catalog_name}.{schema_name}.data_analyst_view
        WHERE base_salary IS NOT NULL
        GROUP BY department
    );
""")

print("✓ Created analyze_operations function")
print("  - Returns: compensation metrics and headcount by department")
print("  - Maintains privacy: no individual employee data exposed")


In [0]:
# Test 1: Performance Analytics
print("\n📊 Test 1: Performance Analytics by Department")
display(spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.analyze_performance()"))

# Test 2: Operations/Compensation Analytics
print("\n📊 Test 2: Compensation Analytics by Department")
display(spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.analyze_operations()"))

## ✅ Governance Foundation Complete

### What We've Accomplished:
1. **Data Classification**: Applied sensitivity labels (Public, Internal, Confidential, Restricted) to all HR tables

2. **Built Classification-Aware Views**: 
   - `data_analyst_view` - Anonymous IDs, year-only dates, full compensation (designed for AI agents)
   - `admin_hr_view` - Complete unmasked access for HR administrators

3. **Configured Group-Based Access**:
   - Set up `hr_data_analysts` group with permissions on views and functions
   - Implemented enterprise-standard permission management
   - Ready for both users and service principals

4. **Implemented Multi-Layer Security**: 
   - Table-level SSN masking (cannot be bypassed)
   - Row-level filtering (Legal department excluded in views)
   - Column-level anonymization (employee IDs → EMP_000001 format)

5. **Built Secure Agent Tools**:
   - `analyze_performance()` - Returns performance ratings and tenure by department
   - `analyze_operations()` - Returns compensation metrics and headcount by department
   - Both functions granted to `hr_data_analysts` group

### Security Architecture Summary:
**Raw Tables** → **Column Masking** → **Views** → **UC Functions** → **AI Agent**
- Column Masking: SSN Protection at table level
- Views: Anonymization + Department Filtering  
- Groups: Permission management (hr_data_analysts)
- UC Functions: Governed access point for agents
- AI Agent: Only sees aggregated, anonymous data

### Next Steps - Lab 2:
1. Build the HR Analyst agent using these secure tools
2. Register the agent model in MLflow
3. Deploy agent as model endpoint
4. Verify agent inherits permissions through group access
5. Test agent with governance-aware queries
6. Evaluate traces to validate security controls

The governance foundation is ready. Let's build the AI agent!